<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [17]:
# Your Code Starts Here
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [18]:
# Your Code Starts Here
def load_quickdraw10(path):
  data = np.load(path)
  X = data['arr_0']
  y = data['arr_1']
  X, y = shuffle(X, y)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                      random_state=42)
  
  return X_train, y_train, X_test, y_test

In [19]:
X_train, y_train, X_test, y_test = load_quickdraw10('../quickdraw10.npz')

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def create_model():
    # create model
    model = Sequential()
    opt = Adam(learning_rate=.001)
    model.add(Dense(32, input_dim=784, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [21]:
import os
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

logdir = os.path.join("logs", "EarlyStopping-Loss")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5)

model= create_model()
model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
   1/2500 [..............................] - ETA: 0s - loss: 88.9758 - accuracy: 0.1562WARNING:tensorflow:From C:\Users\timro\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2500/2500 [==============================] - 2s 616us/step - loss: 2.1097 - accuracy: 0.4431 - val_loss: 1.2942 - val_accuracy: 0.5418
Epoch 2/99
2500/2500 [==============================] - 1s 555us/step - loss: 1.2165 - accuracy: 0.5806 - val_loss: 1.1690 - val_accuracy: 0.6154
Epoch 3/99
2500/2500 [==============================] - 1s 564us/step - loss: 1.1136 - accuracy: 0.6300 - val_loss: 1.0940 - val_accuracy: 0.6544
Epoch 4/99
2500/2500 [==============================] - 1s 577us/step - loss: 1.0500 - accuracy: 0.6647 - val_loss: 1.0892 - val_accuracy: 0.6430
Epoch 5/99
2500/2500 [=========================

In [38]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

wc = MaxNorm(max_value=2)
cpoint = tf.keras.callbacks.ModelCheckpoint("weights_best.h5",
                                            verbose=1,
                                            save_weights_only=True)
def create_model_dropout():
    # create model
    opt = Adam(learning_rate=.001)
    model = Sequential([
    Dense(32,input_dim =784, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dense(32, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [39]:
model= create_model_dropout()
model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[stop, cpoint])

Epoch 1/99
2416/2500 [===========================>..] - ETA: 0s - loss: 1.7976 - accuracy: 0.6243
Epoch 00001: saving model to weights_best.h5
2500/2500 [==============================] - 2s 673us/step - loss: 1.7682 - accuracy: 0.6278 - val_loss: 0.8923 - val_accuracy: 0.7362
Epoch 2/99
2410/2500 [===========================>..] - ETA: 0s - loss: 0.8224 - accuracy: 0.7591
Epoch 00002: saving model to weights_best.h5
2500/2500 [==============================] - 2s 608us/step - loss: 0.8212 - accuracy: 0.7593 - val_loss: 0.7935 - val_accuracy: 0.7683
Epoch 3/99
2476/2500 [============================>.] - ETA: 0s - loss: 0.7492 - accuracy: 0.7831
Epoch 00003: saving model to weights_best.h5
2500/2500 [==============================] - 1s 596us/step - loss: 0.7502 - accuracy: 0.7828 - val_loss: 0.7191 - val_accuracy: 0.7888
Epoch 4/99
2411/2500 [===========================>..] - ETA: 0s - loss: 0.7041 - accuracy: 0.7943
Epoch 00004: saving model to weights_best.h5
2500/2500 [============

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [30]:
# Your Code Starts Here

In [35]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                25120     
_________________________________________________________________
re_lu_2 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_3 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                330       
Total params: 26,506
Trainable params: 26,506
Non-trainable params: 0
_________________________________________________________________


In [36]:
%load_ext tensorboard

In [40]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 12428), started 0:01:55 ago. (Use '!kill 12428' to kill it.)

In [41]:
del model
model

NameError: name 'model' is not defined

In [42]:
model = create_model_dropout()

model.load_weights('weights_best.h5')
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 32)                25120     
_________________________________________________________________
re_lu_6 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                1056      
_________________________________________________________________
re_lu_7 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                330       
Total params: 26,506
Trainable params: 26,506
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.evaluate(X_test, y_test)

625/625 [==============================] - 0s 350us/step - loss: 0.5865 - accuracy: 0.8370


[0.5865088105201721, 0.8369500041007996]

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 

In [ ]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') > .88):   
            self.model.stop_training = True